In [1]:
import os
import torch
from FeatureExtraction import deteksiPerson, kedalamanCitra, ekstrakMetadataCitra, ekstraksiFitur

/home/anugerahsurya/tinggikuenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_path = '../data/Foto.jpg' # Taruh di file gambar yang ditentukan
tinggiobj = 175 #dalam centimeter
visualize = False
dataObjek, kedalamanObjek, metadata, focalLengthpx = ekstraksiFitur(img_path, visualize)

{'ImageWidth': 4640, 'ImageLength': 3472, 'GPSInfo': {'GPSLatitudeRef': 'S', 'GPSLatitude': (6.0, 14.0, 36.75), 'GPSLongitudeRef': 'E', 'GPSLongitude': (106.0, 52.0, 8.79), 'GPSAltitudeRef': b'\x00', 'GPSAltitude': 45.599, 'GPSTimeStamp': (9.0, 6.0, 23.0), 'GPSProcessingMethod': b'ASCII\x00\x00\x00CELLID\x00', 'GPSDateStamp': '2025:05:11'}, 'ResolutionUnit': 2, 'ExifOffset': 218, 'Make': 'Xiaomi', 'Model': 'Redmi Note 9 Pro', 'Orientation': 6, 'DateTime': '2025:05:11 16:06:35', 'YCbCrPositioning': 1, 'XResolution': 72.0, 'YResolution': 72.0, 'ExifVersion': b'0220', 'ComponentsConfiguration': b'\x01\x02\x03\x00', 'ExifImageWidth': 4640, 'ExifImageHeight': 3472, 'DateTimeDigitized': '2025:05:11 16:06:35', 'ExifInteroperabilityOffset': 5371, 'SceneCaptureType': 0, 'MeteringMode': 2, 'LightSource': 21, 'Flash': 16, 'FocalLength': 5.43, 'FocalLengthIn35mmFilm': 25, 'MaxApertureValue': 1.83, 'ExposureBiasValue': 0.0, 'ColorSpace': 1, 'SubsecTime': '049', 'SubsecTimeOriginal': '049', 'SubsecT

/home/anugerahsurya/tinggikuenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
import pandas as pd
def simpanData(dataObjek, kedalamanObjek, metadata, focalLengthpx, tinggiobj):
    # Buat DataFrame
    df = pd.DataFrame(dataObjek)

    # Pisahkan center jadi center_x, center_y
    df[['center_x', 'center_y']] = pd.DataFrame(df['center'].tolist(), index=df.index)
    df.drop(columns='center', inplace=True)

    # Tambahkan kolom kedalamanObjek
    df['kedalamanObjek'] = kedalamanObjek

    # Ambil nilai float dari tensor focalLengthpx
    fl_value = focalLengthpx.item()

    # Tambahkan kolom fl_px dengan nilai focal length yang sama untuk semua baris
    df['fl_px'] = fl_value

    # Tambahkan kolom metadata dengan fallback ke None jika tidak ditemukan
    df['ImageWidth'] = metadata.get('ImageWidth', None)
    df['ImageLength'] = metadata.get('ImageLength', None)
    df['FocalLength'] = metadata.get('FocalLength', None)
    df['FocalLengthIn35mmFilm'] = metadata.get('FocalLengthIn35mmFilm', None)
    df['TinggiAktual'] = tinggiobj
    return df


In [4]:
df = simpanData(dataObjek, kedalamanObjek, metadata, focalLengthpx, tinggiobj)
database = '../data/DatabaseTinggi.csv'

if not os.path.exists(database):
    # File belum ada, simpan baru dengan header
    df.to_csv(database, index=False)
else:
    # File sudah ada, append tanpa header
    df.to_csv(database, mode='a', index=False, header=False)
